In [1]:
# Paso 1: Cargar las librerías necesarias

import torch
from PIL import Image
from transformers import AutoModel, AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig

c:\Users\yera_\Documents\Areas\En proceso\Qualentum-Proyectos\ProspectAI\ProspectAI\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Determinar el dispositivo a usar (CPU o GPU)
device = "cuda" if torch.cuda.is_available() else "cpu"
# device = "cpu"
print(f"Usando dispositivo: {device}")

Usando dispositivo: cuda


In [3]:
img_path = "../prospectos/train/alprazolan/alprazolan.JPG"
# Abre la imagen directamente desde la ruta
image = Image.open(img_path)
prompt = 'Examina el prospecto médico de la imagen y enumera los detalles más importantes relacionados con: - Qué es el medicamento. - Para qué se utiliza el medicamento. - Precauciones antes de tomarlo. - Forma de tomarlo. - Efectos adversos del fármaco. - Conservación del fármaco. - Información adicional.'

In [4]:
model = AutoModel.from_pretrained('openbmb/MiniCPM-Llama3-V-2_5', trust_remote_code=True, torch_dtype=torch.float16)
model = model.to(device=device)

Loading checkpoint shards: 100%|██████████| 7/7 [00:02<00:00,  2.67it/s]


In [5]:
tokenizer = AutoTokenizer.from_pretrained('openbmb/MiniCPM-Llama3-V-2_5', trust_remote_code=True)
model.eval()

MiniCPMV(
  (llm): LlamaForCausalLM(
    (model): LlamaModel(
      (embed_tokens): Embedding(128256, 4096)
      (layers): ModuleList(
        (0-31): 32 x LlamaDecoderLayer(
          (self_attn): LlamaSdpaAttention(
            (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
            (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
            (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
            (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
            (rotary_emb): LlamaRotaryEmbedding()
          )
          (mlp): LlamaMLP(
            (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
            (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
            (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
            (act_fn): SiLU()
          )
          (input_layernorm): LlamaRMSNorm()
          (post_attention_layernorm): LlamaRMSNorm()

In [6]:
img_path = "../prospectos/train/alprazolan/alprazolan.JPG"
# Abre la imagen directamente desde la ruta
image = Image.open(img_path)
question = 'Examina el prospecto médico de la imagen y enumera los detalles más importantes relacionados con: - Qué es el medicamento. - Para qué se utiliza el medicamento. - Precauciones antes de tomarlo. - Forma de tomarlo. - Efectos adversos del fármaco. - Conservación del fármaco. - Información adicional.'
msgs = [{'role': 'user', 'content': question}]

In [7]:
res = model.chat(
    image=image,
    msgs=msgs,
    tokenizer=tokenizer,
    sampling=True, # if sampling=False, beam_search will be used by default
    temperature=0.1,
    # system_prompt='' # pass system_prompt if needed
)
print(res)

c:\Users\yera_\Documents\Areas\En proceso\Qualentum-Proyectos\ProspectAI\ProspectAI\.venv\Lib\site-packages\transformers\models\auto\image_processing_auto.py:513: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(
c:\Users\yera_\Documents\Areas\En proceso\Qualentum-Proyectos\ProspectAI\ProspectAI\.venv\Lib\site-packages\transformers\models\llama\modeling_llama.py:603: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


El prospecto médico de la imagen proporciona información detallada sobre el medicamento Alprazolam, que es un fármaco perteneciente a la familia de los benzodiazepínicos. El medicamento se utiliza comúnmente para tratar síntomas de ansiedad y estrés. Antes de tomar este medicamento, es importante ser consciente de las precauciones, como no utilizarlo si se tiene alergia a Alprazolam o si se está tomando otros medicamentos que puedan interactuar con él. También se indica que el medicamento debe tomarse exactamente según las instrucciones del médico y no se debe sobredosis ni suspender repentinamente.

En cuanto a los efectos adversos, el prospecto menciona posibles reacciones secundarias como somnolencia, debilidad muscular, mareo, visión borrosa, fatiga, irritabilidad, ansiedad aumentada, depresión, confusión, problemas de memoria, alteraciones del lenguaje, problemas de coordinación, temblor, y en casos raros, pensamientos suicidas o intentos de suicidio. Es fundamental informar al mé

## Preparamos el Fine-Tunning

In [ ]:
MODEL="openbmb/MiniCPM-Llama3-V-2_5"  # Modelo preentrenado
DATA="path/to/train_annotations.json"  # Ruta a los datos de entrenamiento
EVAL_DATA="path/to/val_annotations.json"  # Ruta a los datos de validación
LLM_TYPE="llama3"  # Tipo de modelo